# Module 2: Time Series Analysis

This notebook demonstrates time series analysis and forecasting using both sample and real financial data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
import yfinance as yf

sns.set(style="whitegrid")

In [ ]:
# Generate sample time series data
np.random.seed(123)
dates = pd.date_range(start='2020-01-01', periods=120, freq='M')
values = np.random.normal(loc=100, scale=10, size=len(dates))
sample_ts = pd.Series(values, index=dates)

plt.figure(figsize=(10, 5))
plt.plot(sample_ts, label='Sample Time Series')
plt.title('Sample Time Series Data')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
decomposition = seasonal_decompose(sample_ts, model='additive', period=12)
decomposition.plot()
plt.show()

In [ ]:
model = ARIMA(sample_ts, order=(1,1,1))
model_fit = model.fit()
forecast = model_fit.forecast(steps=12)

plt.figure(figsize=(10, 5))
plt.plot(sample_ts, label='Original')
plt.plot(forecast.index, forecast, label='Forecast', color='red')
plt.title('ARIMA Forecast on Sample Data')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

## Real-Time Example: Stock Price Time Series Analysis

Let's analyze and forecast real stock price data using Yahoo Finance.

In [ ]:
stock = yf.download('MSFT', start='2022-01-01', end='2023-01-01')
stock = stock.reset_index()
plt.figure(figsize=(10, 5))
plt.plot(stock['Date'], stock['Close'], label='MSFT Close Price')
plt.title('MSFT Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

In [ ]:
real_ts = pd.Series(stock['Close'].values, index=stock['Date'])
decomp = seasonal_decompose(real_ts, model='additive', period=30)
decomp.plot()
plt.show()

arima_model = ARIMA(real_ts, order=(1,1,1))
arima_fit = arima_model.fit()
real_forecast = arima_fit.forecast(steps=30)

plt.figure(figsize=(10, 5))
plt.plot(real_ts, label='MSFT Close Price')
plt.plot(pd.date_range(real_ts.index[-1], periods=31, freq='D')[1:], real_forecast, label='Forecast', color='red')
plt.title('ARIMA Forecast for MSFT')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

## Holt-Winters (Exponential Smoothing) Forecast on Sample Data

Let's use the Holt-Winters method to forecast the sample time series.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

hw_model = ExponentialSmoothing(sample_ts, trend='add', seasonal='add', seasonal_periods=12)
hw_fit = hw_model.fit()
hw_forecast = hw_fit.forecast(12)

plt.figure(figsize=(10, 5))
plt.plot(sample_ts, label='Original')
plt.plot(hw_forecast.index, hw_forecast, label='Holt-Winters Forecast', color='green')
plt.title('Holt-Winters Forecast on Sample Data')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

## Holt-Winters (Exponential Smoothing) Forecast on Real Stock Data

Now let's apply the Holt-Winters method to the real stock price data.

In [ ]:
hw_model_real = ExponentialSmoothing(real_ts, trend='add', seasonal='add', seasonal_periods=30)
hw_fit_real = hw_model_real.fit()
hw_forecast_real = hw_fit_real.forecast(30)

plt.figure(figsize=(10, 5))
plt.plot(real_ts, label='MSFT Close Price')
plt.plot(pd.date_range(real_ts.index[-1], periods=31, freq='D')[1:], hw_forecast_real, label='Holt-Winters Forecast', color='green')
plt.title('Holt-Winters Forecast for MSFT')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()

## LSTM Forecasting on Time Series Data

Let's use an LSTM neural network for time series forecasting. This requires TensorFlow and Keras.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

# Prepare data for LSTM
scaler = MinMaxScaler()
scaled_sample = scaler.fit_transform(sample_ts.values.reshape(-1, 1))

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 12
X, y = create_sequences(scaled_sample, seq_length)

# Build LSTM model
model = keras.Sequential([
    keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=8, verbose=0)

# Forecast next 12 values
test_input = scaled_sample[-seq_length:]
predictions = []
for _ in range(12):
    pred = model.predict(test_input.reshape(1, seq_length, 1), verbose=0)
    predictions.append(pred[0,0])
    test_input = np.append(test_input, pred)[-seq_length:]
lstm_forecast = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

future_dates = pd.date_range(sample_ts.index[-1] + pd.offsets.MonthEnd(1), periods=12, freq='M')
plt.figure(figsize=(10, 5))
plt.plot(sample_ts, label='Original')
plt.plot(future_dates, lstm_forecast, label='LSTM Forecast', color='purple')
plt.title('LSTM Forecast on Sample Data')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# Prepare real stock data for LSTM
scaled_real = scaler.fit_transform(real_ts.values.reshape(-1, 1))
X_real, y_real = create_sequences(scaled_real, seq_length)

# Build and train LSTM model for real data
model_real = keras.Sequential([
    keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    keras.layers.Dense(1)
])
model_real.compile(optimizer='adam', loss='mse')
model_real.fit(X_real, y_real, epochs=20, batch_size=8, verbose=0)

test_input_real = scaled_real[-seq_length:]
predictions_real = []
for _ in range(30):
    pred = model_real.predict(test_input_real.reshape(1, seq_length, 1), verbose=0)
    predictions_real.append(pred[0,0])
    test_input_real = np.append(test_input_real, pred)[-seq_length:]
lstm_forecast_real = scaler.inverse_transform(np.array(predictions_real).reshape(-1, 1)).flatten()

future_dates_real = pd.date_range(real_ts.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
plt.figure(figsize=(10, 5))
plt.plot(real_ts, label='MSFT Close Price')
plt.plot(future_dates_real, lstm_forecast_real, label='LSTM Forecast', color='purple')
plt.title('LSTM Forecast for MSFT')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.show()